In [4]:
import os
import rasterio as rio
import tifffile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import ImageEnhance, Image
import matplotlib.pyplot as plt
#from osgeo import gdal
import torch
import pickle
#%matplotlib widget

In [2]:
def save_pickle(file, data):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

        
def load_pickle(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [2]:
cbna_data = pd.read_csv('filelist/export_cbna_20220414.csv',sep='\t',low_memory=False)

In [3]:
cbna_releves = cbna_data[['id_releve','id_releve_type','date_releve_deb','x_l93','y_l93']].copy()
cbna_releves['year']=cbna_releves['date_releve_deb'].apply(lambda x: x.split('-')[0] if x==x else 0)
cbna_releves['month']=cbna_releves['date_releve_deb'].apply(lambda x: x.split('-')[1] if x==x else 0)
cbna_releves = cbna_releves.drop_duplicates()

In [4]:
obs_metadata = cbna_releves.reset_index(drop=True)
obs_metadata['id_img'] = obs_metadata.index
obs_metadata.head()

,id_releve,id_releve_type,date_releve_deb,x_l93,y_l93,year,month,id_img
0,9311028,RT,2016-05-23,852114.02,6597167.71,2016,05,0
1,9011945,RT,2010-05-07,881326.00,6544938.00,2010,05,1
2,9163061,RT,2014-06-19,910044.00,6574203.00,2014,06,2
3,9311218,RT,2016-04-29,897967.00,6523258.00,2016,04,3
4,9688247,RT,2014-05-20,893094.00,6524818.00,2014,05,4


In [5]:
#obs_metadata.to_csv('filelist/cbna_simplified.csv', sep=',', encoding='utf-8', index=False)

In [6]:
cbna_sim = pd.read_csv('filelist/cbna_simplified.csv',sep=',',low_memory=False)
cbna_sim.head()

,id_releve,id_releve_type,date_releve_deb,x_l93,y_l93,year,month,id_img
0,9311028,RT,2016-05-23,852114.02,6597167.71,2016,5,0
1,9011945,RT,2010-05-07,881326.00,6544938.00,2010,5,1
2,9163061,RT,2014-06-19,910044.00,6574203.00,2014,6,2
3,9311218,RT,2016-04-29,897967.00,6523258.00,2016,4,3
4,9688247,RT,2014-05-20,893094.00,6524818.00,2014,5,4


In [7]:
cbna_sim.shape

(1595464, 8)

In [8]:
cbna_data.columns.unique()

Index(['id_observation', 'id_releve', 'id_releve_type', 'nom_org_f',
       'lib_releve_methode', 'date_releve_deb', 'nom_obs', 'id_precision',
       'x_l93', 'y_l93', 'surface', 'rec_arbo_pl', 'rec_arbu_pl',
       'rec_sarb_pl', 'rec_herba_pl', 'substr_roch_pl', 'cd_nom', 'cd_ref',
       'nom_reconnu', 'lib_strate', 'abondance', 'comm_taxon', 'lib_det',
       'lib_org_comm', 'meta_original_id_rel', 'meta_original_id_obs',
       'meta_original_bdd', 'valid_reg'],
      dtype='object')

In [9]:
cbna_com = cbna_data.loc[cbna_data['lib_releve_methode'].isin(['Relevé exhaustif habitat','Relevé phytosociologique', 'Relevé exhaustif géogr.'])]
cbna_com.reset_index(drop=True, inplace=True)
cbna_com.shape

(3525072, 28)

In [10]:
res_df = pd.merge(cbna_com[['cd_ref', 'nom_reconnu', 'id_releve']], cbna_sim, on='id_releve').drop_duplicates()
res_df.head()

,cd_ref,nom_reconnu,id_releve,id_releve_type,date_releve_deb,x_l93,y_l93,year,month,id_img
0,98865.0,"Fragaria vesca L., 1753",8988492,RT,2009-07-03,879992.366,6566562.116,2009,7,8
1,86512.0,"Bromopsis erecta (Huds.) Fourr., 1869",8988492,RT,2009-07-03,879992.366,6566562.116,2009,7,8
2,86301.0,"Brachypodium rupestre (Host) Roem. & Schult., ...",8988492,RT,2009-07-03,879992.366,6566562.116,2009,7,8
3,105247.0,"Lathyrus pratensis L., 1753",8988492,RT,2009-07-03,879992.366,6566562.116,2009,7,8
4,115624.0,"Potentilla reptans L., 1753",8988492,RT,2009-07-03,879992.366,6566562.116,2009,7,8


In [11]:
res_df.shape

(3481414, 10)

In [12]:
res_df.dropna(subset = ['cd_ref', 'x_l93', 'y_l93', 'id_releve', 'id_img'], inplace=True)
res_df.reset_index(drop=True, inplace=True)

In [38]:
#res_df.to_csv('filelist/cbna_analysis.csv', sep=',', encoding='utf-8', index=False)

(3481183, 10)

In [17]:
res_norepeat_df = res_df[['id_img', 'id_releve', 'x_l93', 'y_l93']].drop_duplicates()
res_norepeat_df.reset_index(drop=True, inplace=True)
res_norepeat_df.head()

,id_img,id_releve,x_l93,y_l93
0,8,8988492,879992.366000,6.566562e+06
1,13,9609307,929112.688966,6.585048e+06
2,14,9177916,926915.192400,6.579395e+06
3,16,9974388,861730.343400,6.584492e+06
4,23,9950726,848349.322000,6.535917e+06


In [33]:
res_norepeat_df.shape

(162293, 4)

In [ ]:
#res_norepeat_df.to_csv('filelist/cbna_norepeat.csv', sep=',', encoding='utf-8', index=False)

In [54]:
invalid_idx_img = load_pickle('./filelist/invalid_patch.pkl')['invalid_img']

In [55]:
invalid_df = res_norepeat_df.iloc[invalid_idx_img]

In [58]:
invalid_df.shape

(160, 4)

In [ ]:
#invalid_df.to_csv('filelist/cbna_invalid_patches.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
img_idx = invalid_df.id_img.values

In [59]:
res_filtered_df = res_df[~res_df.id_img.isin(img_idx)]

In [61]:
res_filtered_df.shape

(3477371, 10)

In [3]:
#res_filtered_df.to_csv('filelist/cbna_filtered.csv', sep=',', encoding='utf-8', index=False)